In [107]:
import numpy as np
import sympy as sp

In [108]:
# function to check if mask with ith cell dotted matches jth cell of sub
def doesMatch(mask, i, sub, j):
    m = len(mask)
    s = len(sub)
    if (i < 0 or j < 0 or j < i or j - i + m > s or i >= m or j >= s):
        print(mask, ": ", i, j, m, s, "#", i < 0, j < 0, j < i, j - i + m >= s, i >= m, j >= s)
        #print(sub)
        return 0
    else:
        if (mask == sub[j - i: j - i + m]):
            return 1
        else:
            return 0


In [109]:
# test
doesMatch("0", 0, "0011", 2)

0

In [110]:
# function to decide if a cell is able to switch its strategy regarding transition rules
def decideIfTheCellCanFlip(left, center, right, transition_rules):
    n = transition_rules.shape[0]
    for i in range(n):
        if (transition_rules[i][0][0] == left and transition_rules[i][0][1] == center and transition_rules[i][0][2] == right and transition_rules[i][1][0] != center):
            return True
    return False

In [111]:
# general function to calculate probability of conversion of sub-config A to B regarding certain transition rules
def generalCalcProb(A, B, transition_rules = np.array([("000", "1"), ("001", "1"), ("011", "0"), ("100", "1"), ("110", "0"), ("111", "0")])):
    a = len(A)
    b = len(B)
    if (a != b or A[0] != B[0] or A[a - 1] != B[b - 1]):
        return 0
    r, p = sp.symbols("r p")
    r = 1
    for i in range(1, a - 1):    # ignoring first and last cell
        if (A[i] == B[i]):    # must remain on its strategy
            if (decideIfTheCellCanFlip(A[i - 1], A[i], A[i + 1], transition_rules)):    # checking if the ith cell can flip
                r *= (1 - p)
            else:
                r *= 1
        else:
            if (decideIfTheCellCanFlip(A[i - 1], A[i], A[i + 1], transition_rules)):
                r *= p
            else:
                return 0
    return r


In [112]:
# test
print(generalCalcProb("011110", "010110"))

p*(1 - p)**3


In [113]:
def generateAllBinaryStrings(length):
    arr = [""] * 2 ** length
    form = '#0' + str(length + 2) + 'b'
    for i in range(2 ** length):
        arr[i] = format(i, form)[2:]
    return np.array(arr)


In [114]:
# test
print(generateAllBinaryStrings(3))

['000' '001' '010' '011' '100' '101' '110' '111']


In [116]:
# function to calculate the expected difference of number of a list of patterns in a sub-config
def calcExpectedDifference(patterns, sub_config, dot_index, transition_rules = np.array([("000", "1"), ("001", "1"), ("011", "0"), ("100", "1"), ("110", "0"), ("111", "0")])):
    # patterns_max_length = np.max([len(i) for i in patterns[:, 0]])
    k = len(sub_config) - 2
    n = 2 ** k
    binary_strings = generateAllBinaryStrings(k)
    number_of_patterns = patterns.shape[0]
    expected_variation = [0] * number_of_patterns
    for j in range(number_of_patterns):
        #
        total_expectation = 0
        current_number_of_occurrence = doesMatch(patterns[j][0], int(patterns[j][1]), sub_config[1:-1], dot_index - 1)
        for i in range(n):
            total_expectation += generalCalcProb(sub_config, sub_config[0] + binary_strings[i] + sub_config[-1], transition_rules) * (doesMatch(patterns[j][0], int(patterns[j][1]), binary_strings[i], dot_index - 1) - current_number_of_occurrence)
        #
        expected_variation[j] = total_expectation
    return np.array(expected_variation)


In [117]:
# test
pats = np.array([["0", 0], ["11", 1], ["01", 1]])
tr = np.array([("010", "0"), ("011", "0")])
x = calcExpectedDifference(pats, "0011", 2)

In [118]:
x

array([p**2 + p*(1 - p), p*(1 - p), -p**2 - 2*p*(1 - p)], dtype=object)